In [1]:
import requests
import pandas as pd
from urllib.parse import unquote
import plotly.express as px

# URLs and headers
geturl = 'https://www.barchart.com/futures/quotes/CLF25/options/jan-25?futuresOptionsView=merged'
apiurl = 'https://www.barchart.com/proxies/core-api/v1/quotes/get'

getheaders = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
}

getpay = {
    'page': 'all'
}

s = requests.Session()
r = s.get(geturl, params=getpay, headers=getheaders)

headers = {
    'accept': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.barchart.com/futures/quotes/CLF25/options/oct-24?futuresOptionsView=merged',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
    'x-xsrf-token': unquote(unquote(s.cookies.get_dict()['XSRF-TOKEN']))
}

payload = {
    'fields': 'symbol,contractSymbol,lastPrice,priceChange,openPrice,highPrice,lowPrice,previousPrice,volume,openInterest,tradeTime,symbolCode,symbolType,hasOptions',
    'list': 'futures.contractInRoot',
    'root': 'CL',
    'meta': 'field.shortName,field.type,field.description',
    'hasOptions': 'true',
    'raw': '1'
}

r = s.get(apiurl, params=payload, headers=headers)
j = r.json()

# Convert JSON to DataFrame
df = pd.json_normalize(j['data'])  # Adjust the key based on the actual structure of your JSON
df = df.drop(0).reset_index(drop=True)
# Print DataFrame
print(df)
first_10_symbols = df['symbol'].iloc[:10].tolist()
print(first_10_symbols)


# Clean the 'lastPrice' column: remove non-numeric characters and convert to float
df['lastPrice'] = df['lastPrice'].str.replace('s', '', regex=False).astype(float)

# Plot using Plotly
fig = px.scatter(df, x='symbol', y='lastPrice', 
                 title='Forward Curve',
                 labels={'symbol': 'Contract Symbol', 'lastPrice': 'Last Price'},)

# Show the plot
fig.show()

    symbol   contractSymbol lastPrice priceChange openPrice highPrice  \
0    CLV25  CLV25 (Oct '25)     63.16       +0.48     62.74     63.18   
1    CLX25  CLX25 (Nov '25)     62.80       +0.40     62.30     62.93   
2    CLZ25  CLZ25 (Dec '25)     62.37       +0.39     61.92     62.48   
3    CLF26  CLF26 (Jan '26)     62.10       +0.40     61.67     62.18   
4    CLG26  CLG26 (Feb '26)     61.91       +0.39     61.46     61.99   
..     ...              ...       ...         ...       ...       ...   
120  CLV35  CLV35 (Oct '35)    61.51s       +0.06      0.00     61.51   
121  CLX35  CLX35 (Nov '35)    61.52s       +0.06      0.00     61.52   
122  CLZ35  CLZ35 (Dec '35)    61.51s       +0.06      0.00     61.51   
123  CLF36  CLF36 (Jan '36)    61.44s       +0.06      0.00     61.44   
124  CLG36  CLG36 (Feb '36)    61.38s       +0.06     62.00     62.00   

    lowPrice previousPrice  volume openInterest  ... raw.openPrice  \
0      62.74         62.68      58       17,577  ... 

In [2]:
import requests
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import datetime


# Base URL format
base_url = "https://www.barchart.com/futures/quotes/{}/options/"

# Headers to avoid 403 errors
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.google.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

# List to store extracted data
data = []

# Scrape data for each contract
for contract in first_10_symbols:
    url = base_url.format(contract)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        toolbar = soup.find('div', class_='row bc-options-toolbar__second-row')

        if toolbar:
            columns = toolbar.find_all('div', class_='column')
        

            days_to_expiry = columns[0].get_text(strip=True)
            implied_volatility = columns[1].get_text(strip=True)

            # Extract only numerical values
            expiration_date = days_to_expiry.split()[3][2:]
            days_to_expiry = int(days_to_expiry.split()[0])
            
            implied_volatility = float(implied_volatility.split(":")[1].replace("%", "").strip())

            # Get last price from df
            last_price = df.loc[df["symbol"] == contract, "lastPrice"].values[0]

            # Append data
            data.append([contract, last_price,expiration_date, days_to_expiry, implied_volatility])
        else:
            print(f"Could not find options toolbar for {contract}.")
    else:
        print(f"Failed to fetch data for {contract}. Status code: {response.status_code}")

# Create DataFrame
futures_df = pd.DataFrame(data, columns=["Contract", "Last Price","Expiration Date", "Options Days to Expiry", " Futures Implied Volatility"])


# Display the DataFrame
print(futures_df)


Could not find options toolbar for CLV25.
  Contract  Last Price Expiration Date  Options Days to Expiry  \
0    CLX25       62.80        10/16/25                      24   
1    CLZ25       62.37        11/17/25                      56   
2    CLF26       62.10        12/16/25                      85   
3    CLG26       61.91        01/14/26                     114   
4    CLH26       61.80        02/17/26                     148   
5    CLJ26       61.79        03/17/26                     176   
6    CLK26       61.80        04/16/26                     206   
7    CLM26       61.69        05/14/26                     234   
8    CLN26       61.64        06/16/26                     267   

    Futures Implied Volatility  
0                        26.76  
1                        28.33  
2                        29.29  
3                        29.49  
4                        29.70  
5                        29.78  
6                        29.73  
7                        29.58  


In [3]:
import requests
from urllib.parse import unquote
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
}

def main(url, symbols):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url[:25])
        req.headers.update(
            {'X-XSRF-TOKEN': unquote(r.cookies.get_dict()['XSRF-TOKEN'])})
        

        results = {}
        for i, symbol in enumerate(symbols, start=1):
            params = {
                "symbol": symbol,
                "list": "futures.options",
                "fields": "strike,openPrice,highPrice,lowPrice,lastPrice,priceChange,bidPrice,askPrice,volume,openInterest,premium,tradeTime,longSymbol,optionType,symbol",
                "orderBy": "strike",
                "orderDir": "asc",
                "meta": "field.shortName,field.description,field.type",
                "futureOptions": "true",
                "noPagination": "true",
                "showExpandLink": "false"
            }
            
            r = req.get(url, params=params).json()
            
            df = pd.DataFrame(r['data'])
            df['strike'] = df['strike'].astype(str).str.replace(r'[CP]', '', regex=True)
            df['volume'] = pd.to_numeric(df['volume'], errors='coerce').fillna(0).astype(int)
            df['openInterest'] = pd.to_numeric(df['openInterest'], errors='coerce').fillna(0).astype(int)
            df['lastPrice'] = df['lastPrice'].str.replace(r'[a-zA-Z]', '', regex=True).astype(float)

            
            
            calls_df = df[df['optionType'] == 'Call']
            puts_df = df[df['optionType'] == 'Put']
            
            results[symbol] = (calls_df, puts_df)

        return results
                

# Specify the URL and the output file path
url = 'https://www.barchart.com/proxies/core-api/v1/quotes/get?'



# Call the main function with the list of symbols and output file
results = main(url, first_10_symbols)



In [4]:
symbols = results.keys()
print(list(symbols))

for symbol, (calls_df, puts_df) in results.items():
    print(f"Symbol: {symbol}")
    
    print("DataFrame Columns:")
    print(calls_df.columns)  # Prints column names of calls DataFrame
    
    print("Calls DataFrame Sample:")
    print(calls_df.head())  # Prints first five rows of calls DataFrame
    
    print("Puts DataFrame Sample:")
    print(puts_df.head())  # Prints first five rows of puts DataFrame


['CLQ25', 'CLU25', 'CLV25', 'CLX25', 'CLZ25', 'CLF26', 'CLG26', 'CLH26', 'CLJ26', 'CLK26']
Symbol: CLQ25
DataFrame Columns:
Index(['strike', 'openPrice', 'highPrice', 'lowPrice', 'lastPrice',
       'priceChange', 'bidPrice', 'askPrice', 'volume', 'openInterest',
       'premium', 'tradeTime', 'longSymbol', 'optionType', 'symbol'],
      dtype='object')
Calls DataFrame Sample:
   strike openPrice highPrice lowPrice  lastPrice priceChange bidPrice  \
17  30.00       N/A     38.38    38.38      38.38       +0.05      N/A   
38  40.00       N/A     28.38    28.38      28.38       +0.05     0.25   
41  41.00       N/A     27.38    27.38      27.38       +0.05      N/A   
54  47.00       N/A     21.38    21.38      21.38       +0.05      N/A   
57  48.00       N/A     20.38    20.38      20.38       +0.05      N/A   

   askPrice  volume  openInterest    premium tradeTime longSymbol optionType  \
17      N/A       0            29  38,380.00  07/09/25  CLQ5|300C       Call   
38      N/A    

In [5]:
def get_option_data(results, symbol, strike, option_type):
    """
    Retrieves the row of options data for a given symbol, strike price, and option type.

    Parameters:
    - results (dict): Dictionary containing options data.
    - symbol (str): The symbol to filter.
    - strike (float): The strike price to filter.
    - option_type (str): Either 'call' or 'put'.

    Returns:
    - DataFrame: The filtered row(s) if found, otherwise None.
    """
    if symbol not in results:
        print(f"Symbol '{symbol}' not found.")
        return None

    calls_df, puts_df = results[symbol]

    # Select the appropriate DataFrame
    df = calls_df if option_type.lower() == "call" else puts_df if option_type.lower() == "put" else None
    
    if df is None:
        print("Invalid option type. Choose 'call' or 'put'.")
        return None

    if "strike" not in df.columns:
        print("Strike column not found in the DataFrame.")
        return None

    # Filter by strike price
    filtered_row = df[df["strike"] == strike]

    if filtered_row.empty:
        print(f"No data found for {symbol} {option_type.upper()} with Strike: {strike}")
        return None

    return filtered_row

# Example usage:
result = get_option_data(results, "CLK25", "64.00", "put")  
if result is not None:
    print(result)


Symbol 'CLK25' not found.


In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import brentq

# Optimized Binomial Tree Model for American Options
def binomial_price_american(F, K, T, r, sigma, option_type, steps=50):  # Reduced steps to 50
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))  
    d = 1 / u  
    p = (np.exp(r * dt) - d) / (u - d)  

    # Terminal stock prices
    ST = np.array([F * (u ** j) * (d ** (steps - j)) for j in range(steps + 1)])
    
    # Terminal option values
    if option_type == "C":
        option_values = np.maximum(ST - K, 0)
    else:
        option_values = np.maximum(K - ST, 0)

    # Backward induction (vectorized)
    discount = np.exp(-r * dt)
    for i in range(steps - 1, -1, -1):
        ST = ST[:-1] * u  # Move one step back
        early_exercise = np.maximum(ST - K, 0) if option_type == "C" else np.maximum(K - ST, 0)
        option_values = np.maximum(early_exercise, discount * (p * option_values[1:] + (1 - p) * option_values[:-1]))

    return option_values[0]

# Function to calculate implied volatility
def implied_volatility(option_price, F, K, T, r, option_type):
    try:
        return brentq(lambda sigma: binomial_price_american(F, K, T, r, sigma, option_type) - float(option_price), 
                      0.001, 3.0, xtol=1e-6)
    except ValueError:
        return np.nan  # Return NaN if IV cannot be solved

# Assume a constant risk-free rate
risk_free_rate = 0.03  

# Merging futures data with options data
merged_data = []

for symbol, (calls_df, puts_df) in results.items():
    future_row = futures_df[futures_df['Contract'] == symbol]
    if future_row.empty:
        continue  
    
    F = future_row['Last Price'].values[0]  
    T = future_row['Options Days to Expiry'].values[0] / 365.0  
    
    # Process Call Options
    for _, row in calls_df.iterrows():
        K = float(row['strike'])  
        option_price = float(row['lastPrice'])
        iv = implied_volatility(option_price, F, K, T, risk_free_rate, "C")
        merged_data.append((symbol, "Call", K, T, option_price, iv))
    
    # Process Put Options
    for _, row in puts_df.iterrows():
        K = float(row['strike'])  
        option_price = float(row['lastPrice'])
        iv = implied_volatility(option_price, F, K, T, risk_free_rate, "P")
        merged_data.append((symbol, "Put", K, T, option_price, iv))

# Convert to DataFrame
implied_vol_df = pd.DataFrame(merged_data, columns=["Symbol", "Type", "Strike", "Time to Expiry", "Market Price", "Implied Volatility"])

# Display Results
print(implied_vol_df.head())


  Symbol  Type  Strike  Time to Expiry  Market Price  Implied Volatility
0  CLQ25  Call    30.0        0.021918         38.38            2.534015
1  CLQ25  Call    40.0        0.021918         28.38            1.689640
2  CLQ25  Call    41.0        0.021918         27.38            1.617503
3  CLQ25  Call    47.0        0.021918         21.38            1.219387
4  CLQ25  Call    48.0        0.021918         20.38            1.158146


In [7]:
import plotly.graph_objects as go
import numpy as np

# Loop through unique symbols
for symbol in implied_vol_df['Symbol'].unique():
    # Filter the data for the specific symbol
    symbol_data = implied_vol_df[implied_vol_df['Symbol'] == symbol]
    
    # Get the current futures price (F) for the symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    F = future_row['Last Price'].values[0]  # Futures price
    print(symbol, F)
    
    # Find the ATM strike (closest to F)
    atm_strike = min(symbol_data['Strike'], key=lambda x: abs(x - F))
    
    # Limit strikes to ±15 from ATM
    symbol_data = symbol_data[symbol_data['Strike'].between(atm_strike - 15, atm_strike + 15)]
    
    # Separate Call and Put data
    call_data = symbol_data[symbol_data['Type'] == 'Call']
    put_data = symbol_data[symbol_data['Type'] == 'Put']
    
    # Create figure
    fig = go.Figure()

    # Plot for Call Options
    fig.add_trace(go.Scatter(x=call_data['Strike'], y=call_data['Implied Volatility'], 
                             mode='markers', name=f'Call Options - {symbol}', 
                             marker=dict(symbol='circle', color='blue', size=8, opacity=0.7)))

    # Plot for Put Options
    fig.add_trace(go.Scatter(x=put_data['Strike'], y=put_data['Implied Volatility'], 
                             mode='markers', name=f'Put Options - {symbol}', 
                             marker=dict(symbol='x', color='red', size=8, opacity=0.7)))

    # Add vertical line for Current Price (F)
    fig.add_trace(go.Scatter(x=[F, F], y=[0, max(symbol_data['Implied Volatility'])], 
                             mode='lines', name=f'Current Price (F) - {symbol}', 
                             line=dict(color='green', dash='dash')))

    # Check for Skew and Smile
    atm_call_iv = call_data[call_data['Strike'] == atm_strike]['Implied Volatility'].values
    atm_put_iv = put_data[put_data['Strike'] == atm_strike]['Implied Volatility'].values

    if len(atm_call_iv) > 0 and len(atm_put_iv) > 0:
        atm_call_iv = atm_call_iv[0]
        atm_put_iv = atm_put_iv[0]

        if atm_put_iv > atm_call_iv:
            skew_type = "Left Skew (Put IV > Call IV)"
        elif atm_put_iv < atm_call_iv:
            skew_type = "Right Skew (Call IV > Put IV)"
        else:
            skew_type = "Balanced Smile"
        
        print(f"{symbol} - {skew_type}")

        # Add Skew Type Annotation to the Plot
        fig.add_annotation(
            x=atm_strike, 
            y=max(symbol_data['Implied Volatility']), 
            text=skew_type, 
            showarrow=False, 
            font=dict(size=12, color="black"),
            align="center",
            bordercolor="black",
            borderwidth=1,
            borderpad=4,
            bgcolor="white",
        )

    # Update layout
    fig.update_layout(
        title=f'Implied Volatility Smile for {symbol}',
        xaxis_title='Strike Price',
        yaxis_title='Implied Volatility',
    )
    
    # Show the plot
    fig.show()


CLQ25 68.28
CLQ25 - Left Skew (Put IV > Call IV)


CLU25 66.93
CLU25 - Right Skew (Call IV > Put IV)


CLV25 65.6
CLV25 - Left Skew (Put IV > Call IV)


CLX25 64.74
CLX25 - Balanced Smile


CLZ25 64.0
CLZ25 - Balanced Smile


CLF26 63.75
CLF26 - Balanced Smile


CLG26 63.5
CLG26 - Balanced Smile


CLH26 63.33
CLH26 - Left Skew (Put IV > Call IV)


CLJ26 63.2


CLK26 63.14


In [8]:
import plotly.express as px

fig = px.line(implied_vol_df, x="Time to Expiry", y="Implied Volatility", color="Symbol",
              markers=True, title="Implied Volatility Term Structure")

fig.show()


In [9]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Initialize lists to store data for the surface plot
strike_prices = []
contract_months = []
implied_vols = []

# Loop through unique contract symbols (months)
for symbol in implied_vol_df['Symbol'].unique():
    # Filter the data for the specific symbol
    symbol_data = implied_vol_df[implied_vol_df['Symbol'] == symbol]
    
    # Get the current futures price (F) for the symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    F = future_row['Last Price'].values[0]  # Futures price
    
    # Find the ATM strike (closest to F)
    atm_strike = min(symbol_data['Strike'], key=lambda x: abs(x - F))
    
    # Limit strikes to ±15 from ATM
    symbol_data = symbol_data[symbol_data['Strike'].between(atm_strike - 10, atm_strike + 10)]
    
    # Append data to lists
    for _, row in symbol_data.iterrows():
        strike_prices.append(row['Strike'])
        contract_months.append(symbol)
        implied_vols.append(row['Implied Volatility'])

# Convert to NumPy arrays for surface plotting
X = np.array(strike_prices)
Y = np.array(contract_months)  # This is categorical; it will be converted to numerical
Z = np.array(implied_vols)

# Create the surface plot
fig = go.Figure(data=[go.Mesh3d(
    x=X,
    y=pd.factorize(Y)[0],  # Convert categorical contract months to numerical indices
    z=Z,
    colorbar_title='Implied Volatility',
    colorscale='Viridis',
    opacity=0.8
)])

# Update layout for better visibility
fig.update_layout(
    title='Implied Volatility Surface',
    width=1000, height=700,  # Increase figure size
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Contract Month',
        zaxis_title='Implied Volatility',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.0))  # Adjust viewing angle
    )
)

# Show the plot
fig.show()


In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Function to adjust implied volatility for Asian options
def adjust_iv_for_asian(iv, n):
    """Adjusts the implied volatility for an Asian option using averaging adjustment."""
    return iv * np.sqrt(2 / (n + 1))

# Asian option pricing (Turnbull-Wakeman approximation for arithmetic average price options)
def asian_option_price(F, K, T, r, sigma_asian, option_type):
    """Computes the price of an Asian option using Turnbull-Wakeman approximation."""
    sigma_adj = sigma_asian * np.sqrt(3 / (2 * T))
  # Adjusted volatility
    d1 = (np.log(F / K) + (0.5 * sigma_adj ** 2) * T) / (sigma_adj * np.sqrt(T))
    d2 = d1 - sigma_adj * np.sqrt(T)
    
    if option_type == "Call":  # Call option
        return np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == "Put":  # Put option
        return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("Invalid option type: must be 'C' or 'P'")

# Assume an averaging period (e.g., 30 days for a monthly Asian option)
n = 30  # Number of averaging periods
risk_free_rate = 0.03  # Example risk-free rate (3%)


# Creating a new DataFrame for Asian option calculations
asian_option_data = []

for _, row in implied_vol_df.iterrows():
    symbol = row["Symbol"]
    option_type = row["Type"]
    K = row["Strike"]
    T = row["Time to Expiry"]
    iv = row["Implied Volatility"]
    
    # Find the corresponding futures price for the option's underlying symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    if future_row.empty:
        continue  # Skip if no matching future found
    
    F = future_row['Last Price'].values[0]  # Assign actual futures price
    
    # Adjust the implied volatility for Asian options
    sigma_asian = adjust_iv_for_asian(iv, n)
    
    # Calculate the Asian option price
    asian_price = asian_option_price(F, K, T, risk_free_rate, sigma_asian, option_type)
    
    # Store results
    asian_option_data.append((symbol, option_type, K, T, sigma_asian, asian_price))


# Convert to DataFrame
asian_option_df = pd.DataFrame(asian_option_data, columns=[
    "Symbol", "Type", "Strike", "Time to Expiry", "Adjusted Asian IV", "Asian Option Price"
])

# Display Results
print("=== Implied Volatility from American Options ===")
print(implied_vol_df)
print("\n=== Asian Option Adjusted IV & Price ===")
print(asian_option_df)


=== Implied Volatility from American Options ===
     Symbol  Type  Strike  Time to Expiry  Market Price  Implied Volatility
0     CLQ25  Call    30.0        0.021918         38.38            2.534015
1     CLQ25  Call    40.0        0.021918         28.38            1.689640
2     CLQ25  Call    41.0        0.021918         27.38            1.617503
3     CLQ25  Call    47.0        0.021918         21.38            1.219387
4     CLQ25  Call    48.0        0.021918         20.38            1.158146
...     ...   ...     ...             ...           ...                 ...
2359  CLK26   Put    61.0        0.769863          5.36            0.319900
2360  CLK26   Put    64.0        0.769863          6.72            0.312916
2361  CLK26   Put    65.0        0.769863          7.23            0.309919
2362  CLK26   Put    71.0        0.769863         10.85            0.301673
2363  CLK26   Put    72.0        0.769863         11.55            0.301979

[2364 rows x 6 columns]

=== Asian Opt

In [11]:
def get_asian_option_data(asian_option_df, symbol, strike, option_type):
    """
    Retrieves the row of Asian options data for a given symbol, strike price, and option type.

    Parameters:
    - asian_option_df (pd.DataFrame): DataFrame containing Asian options data.
    - symbol (str): The symbol to filter.
    - strike (float): The strike price to filter.
    - option_type (str): Either 'Call' or 'Put'.

    Returns:
    - DataFrame: The filtered row(s) if found, otherwise None.
    """
    # Filter based on symbol, strike, and option type
    filtered_row = asian_option_df[
        (asian_option_df["Symbol"] == symbol) &
        (asian_option_df["Strike"] == strike) &
        (asian_option_df["Type"] == option_type)
    ]

    if filtered_row.empty:
        print(f"No Asian option data found for {symbol} {option_type} with Strike: {strike}")
        return None

    return filtered_row

# Example usage:
result = get_asian_option_data(asian_option_df, "CLK25", 77.00, "Put")
if result is not None:
    print(result)


No Asian option data found for CLK25 Put with Strike: 77.0
